In [1]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time

from bitalino import BITalino
import biofeatures

In [ ]:
macAddress = "/dev/tty.BITalino-DevB"

running_time = 120
    
acqChannels = [1,2]
samplingRate = 100
nSamples = 100

# Calculate features every 10 seconds
feature_interval = 10
feature_time = 0

# Connect to BITalino
device = BITalino(macAddress)

start = time.time()
end = time.time()

inflating = False
inflating_start_time = time.time()

data = []
features = []
trends = {}


In [ ]:
# Start Acquisition
device.start(samplingRate, acqChannels)

try:
    while (end - start) < running_time:
        start_loop = time.time()
        
        new_samples = device.read(nSamples)
        new_signal = [s[-1] for s in new_samples]
    
        data += new_signal
        
        if inflating and (time.time() - inflating_start_time) > 5:
            client.send_message("/actuator/1/inflate", -50.0)
            print("deflating")
            inflating = False
        
        if feature_time > feature_interval:
            data = data[-samplingRate*5*60:]

            intervals = biofeatures.r_peak_intervals(data, samplingRate)

            new_features = biofeatures.hrv_features(intervals)
            features.append(new_features)
            print(new_features)
            
            if (end - start) > 1 * 60:
                
                for key in features[0].keys():
                    trends[key] = biofeatures.detect_trend([f[key] for f in features])
        
                if trends["rmssd"] < 0:
                    print("detected increase in anxiety")
                    
                if not inflating:
                    client.send_message("/actuator/1/inflate", 50.0)
                    print("inflating")
                    inflating_start_time = time.time()
                    inflating = True
                    
            
            feature_time = 0
        
        
        feature_time += time.time() - end
        end = time.time()

        end_loop = time.time()

except:
    print("Data processing crashed")

finally:   
    # Stop acquisition
    device.stop()
    
    # Close connection
    device.close()

In [ ]:
deflating = False

def print_volume_handler(unused_addr, args, volume):
    global deflating, deflateStartTime
    
    print("[{0}] ~ {1}".format(args, volume))
    
    if (volume > 1090 or deflating):
        client.send_message("/actuator/1/inflate", -80.0)   # Send float message between -100 and 100
        
        if deflating == False:
            deflating = True
            deflateStartTime = time.time()
        
        elif time.time() - deflateStartTime > 5:
            deflating = False
            
    elif deflating == False:
        client.send_message("/actuator/1/inflate", 80.0)   # Send float message between -100 and 100


deflateStartTime = time.time()
    
ip = "192.168.0.106"
port = 32000

client = SimpleUDPClient(ip, port)  # Create client
    
dispatcher = dispatcher.Dispatcher()
dispatcher.map("/sensor/pressure", print_volume_handler, "Pressure")

server = osc_server.ThreadingOSCUDPServer(("192.168.0.106", 31000), dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()

In [ ]:
resp_data = []
last_update = time.time()

update_freq = 0.5

riot_ip = '192.168.1.2'
riot_port = 8888
actuator_port = 12000
actuator_ip = '192.168.0.103'

client = SimpleUDPClient(actuator_ip, actuator_port) 

def process_riot_data(unused_addr, *values):
    global resp_data, last_update, client
        
    new_data = values[12]
    resp_data.append(new_data)
        
    if len(resp_data) > 200*10 and time.time() - last_update > update_freq:
        last_int, breathe_in = biofeatures.calc_resp_intervals(resp_data, last_breath = True)
                
        if breathe_in:
            print("Breathing in")
            client.send_message("/actuator/inflate", 100.0)
        else:
            print("Breathing out")
            client.send_message("/actuator/inflate", -100.0)
        
        last_update = time.time()
    
    # only save the last 5 min of data
    if len(resp_data) > 200 * 60 * 5:
        resp_data = resp_data[-200*60*5:]

In [ ]:
riot_dispatcher = dispatcher.Dispatcher()
riot_dispatcher.map("/*/raw", process_riot_data)

server = osc_server.ThreadingOSCUDPServer((riot_ip, riot_port), riot_dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()